<a href="https://colab.research.google.com/github/bolulka/NeuralNetworks/blob/main/lab7/lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лаб 7.1
Задание 1.  (из  темы  7.2 – 7.3):

Выполните базовую версию программы обучения сверточной нейронной сети распознавания изображений.

Попытайтесь улучшить качество обучения сети путем изменения различных гиперпараметров.

Сохраните обученую сверточную нейронную сеть распознавания изображений.

Задание 2.  (из  темы  7.2):

Загрузите сохраненную сверточную нейронную сеть распознавания изображений.

Задание 3  (из темы 7.3):

Обучите сверточную нейронную сеть распознаванию цифр.

In [1]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

# Задаем seed для повторяемости результатов
numpy.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10

# Количество эпох для обучения
# nb_epoch = 15
# nb_epoch = 25
nb_epoch = 30
'''
15 - 76.22%
25 - 76.46%
30 - 76.98%
'''

# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Создаем последовательную модель
model = Sequential()
# Формирование вектора, отвечающего за размерность входных данных: 
# либо (кол-во каналов, ширина, высота), либо (ширина, высота, кол-во каналов) 
# (в зависимости от значения параметра "image_data_format" в файле keras.json)
shape_vector = X_train.shape[1:]
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=shape_vector, activation='relu'))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))

'''
with - 62.13%
without - 61.24%
'''
# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
'''
0.01 - 52.16%
0.02 - 49.79%
0.05 - 10.00%
0.1 - 10.00%
'''
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=1)

# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("cifar10_model.json", "w")
json_file.write(model_json)
json_file.close()

model.save_weights("cifar10_model.h5")
print("Сохранили Model")

'''
Точность работы на тестовых данных: 76.64%
'''

170500096/170498071 [==============================] - 2s 0us/step
Epoch 1/30
1407/1407 [==============================] - 40s 5ms/step - loss: 1.9823 - accuracy: 0.2641 - val_loss: 1.4028 - val_accuracy: 0.4882
Epoch 2/30
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3655 - accuracy: 0.5081 - val_loss: 1.1127 - val_accuracy: 0.5964
Epoch 3/30
1407/1407 [==============================] - 7s 5ms/step - loss: 1.1600 - accuracy: 0.5880 - val_loss: 1.2631 - val_accuracy: 0.5654
Epoch 4/30
1407/1407 [==============================] - 7s 5ms/step - loss: 1.0243 - accuracy: 0.6361 - val_loss: 0.8611 - val_accuracy: 0.6992
Epoch 5/30
1407/1407 [==============================] - 7s 5ms/step - loss: 0.9268 - accuracy: 0.6725 - val_loss: 0.8130 - val_accuracy: 0.7182
Epoch 6/30
1407/1407 [==============================] - 7s 5ms/step - loss: 0.8687 - accuracy: 0.6930 - val_loss: 0.7945 - val_accuracy: 0.7232
Epoch 7/30
1407/1407 [==============================] - 7s 5ms/step 

'\nТочность работы на тестовых данных: 76.64%\n'

In [ ]:
from keras.models import model_from_json
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.utils import np_utils

# Загружаем данные об архитектуре сети из файла json
json_filename = "cifar10_model.json"
with open(json_filename, "r") as json_file:
    loaded_model_json = json_file.read()
json_file.close()

# Создаем модель на основе загруженных данных
model = model_from_json(loaded_model_json)

# Загружаем веса в модель
h5_filename = "cifar10_model.h5"
model.load_weights(h5_filename)

# Перед использованием загруженной нейронной сети необходимо её скомпилировать
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# Вывести начальные характеристики нейросети
print(model.summary())

# Данные для тестирования загруженной нейронной сети
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

'''
Точность работы на тестовых данных: 76.98%
'''

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)         

'\nТочность работы на тестовых данных: 76.98%\n'

In [ ]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.backend import image_data_format

# Устанавливаем seed для повторяемости результатов
numpy.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Формирование вектора размерности (зависит от параметра из файла keras.json)
input_shape = ((1, *X_train.shape[1:]) if 
               image_data_format() == 'channels_first' else 
               (*X_train.shape[1:], 1))

# Преобразование размерности изображений
X_train = X_train.reshape(X_train.shape[0], *input_shape)
X_test = X_test.reshape(X_test.shape[0], *input_shape)

# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# Создаем последовательную модель
model = Sequential()

model.add(Conv2D(75, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(100, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Компилируем модель
model.compile(loss="categorical_crossentropy", optimizer="adam", 
              metrics=["accuracy"])

print(model.summary())

# Обучаем сеть
model.fit(X_train, Y_train, batch_size=200, epochs=1, validation_split=0.2, 
          verbose=1)

# Оцениваем качество обучения сети на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

'''
accuracy: 0.9164
val_accuracy: 0.9825
Точность работы на тестовых данных: 98.29%
'''

11493376/11490434 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 24, 24, 75)        1950      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 75)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 75)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 100)         187600    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 100)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 4, 100)         0         
_______________________________________________________

'\naccuracy: 0.9164\nval_accuracy: 0.9825\nТочность работы на тестовых данных: 98.29%\n'